In [21]:
#Dependencies
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt

## Reflect Tables into SQLAlchemy ORM
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import inspect


In [18]:
#Create engine 
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

#Declare Base using automap_base()
Base = automap_base()

##############
# reflect an existing database into a new model
Base.prepare(engine, reflect=True)

# reflect the tables
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [19]:
 # Save references to each table
 Measurement =  Base.classes.measurement
 Station = Base.classes.station

In [20]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [22]:
#Determine what attributes are in the class
###Use inspector to get 'measurement' attributes, 'station' attributes
inspector = inspect(engine)
inspector.get_table_names()
columns = inspector.get_coloumns('')

['measurement', 'station']

In [26]:
#Load Measurement and Station into panda dataframes
stmt = session.query(Measurement).statement
Measurement_df = pd.read_sql_query(stmt, session.bind)
print("Measurement_df")
print(Measurement_df.head())

stmt = session.query(Station).statement
Station_df = pd.read_sql_query(stmt, session.bind)
print("Station_df")
print(Station_df.head())


Measurement_df
   id      station        date  prcp  tobs
0   1  USC00519397  2010-01-01  0.08  65.0
1   2  USC00519397  2010-01-02  0.00  63.0
2   3  USC00519397  2010-01-03  0.00  74.0
3   4  USC00519397  2010-01-04  0.00  76.0
4   5  USC00519397  2010-01-06   NaN  73.0
Station_df
   id      station                                    name  latitude  \
0   1  USC00519397                    WAIKIKI 717.2, HI US   21.2716   
1   2  USC00513117                    KANEOHE 838.1, HI US   21.4234   
2   3  USC00514830  KUALOA RANCH HEADQUARTERS 886.9, HI US   21.5213   
3   4  USC00517948                       PEARL CITY, HI US   21.3934   
4   5  USC00518838              UPPER WAHIAWA 874.3, HI US   21.4992   

   longitude  elevation  
0  -157.8168        3.0  
1  -157.8015       14.6  
2  -157.8374        7.0  
3  -157.9751       11.9  
4  -158.0111      306.6  


Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results

# Calculate the date 1 year ago from the last data point in the database

# Perform a query to retrieve the data and precipitation scores

# Save the query results as a Pandas DataFrame and set the index to the date column

# Sort the dataframe by date

# Use Pandas Plotting with Matplotlib to plot the data

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data

In [ ]:
# Design a query to show how many stations are available in this dataset?

In [ ]:
 # What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram